# Raport Techniczny: Etap 2 – Ekstrakcja Cech i Parametryzacja Obrazu

**Autor:** Julia Diadia
**Cel etapu:** Transformacja obrazów rastrowych i masek binarnych (przygotowanych w Etapie 1) na zbiór danych numerycznych (wektory cech) umożliwiający klasyfikację obiektów przy użyciu algorytmów uczenia maszynowego.

---

### 1. Analiza danych wejściowych i definicja klas

Na podstawie struktury katalogów otrzymanych po Etapie 1 (`Output` oraz `Output_masks`), zidentyfikowano trzy główne klasy obiektów podlegających analizie:
1.  **RBC (Red Blood Cells):** Erytrocyty.
2.  **WBC (White Blood Cells):** Leukocyty (klasa zbiorcza).
3.  **Platelets:** Płytki krwi (Trombocyty).

Ze względu na morfologię masek RBC (występowanie artefaktów w postaci „dziur” w centrum krwinki), w procesie ekstrakcji cech kształtu zastosowano algorytmy uwzględniające jedynie **kontur zewnętrzny** obiektu.

---

### 2. Metodyka ekstrakcji cech

W celu stworzenia uniwersalnego wektora cech, zastosowano analizę w trzech domenach: koloru, kształtu i tekstury. Łącznie wygenerowano wektor opisujący każdy obiekt.

#### A. Analiza Koloru (Color Features)
Zastosowano obliczenie średniej intensywności pikseli w przestrzeni barw **BGR** wewnątrz obszaru wyznaczonego przez maskę segmentacji.
* **Uzasadnienie:** Leukocyty charakteryzują się silnym wybarwieniem jądra (kolor fioletowy/niebieski), podczas gdy erytrocyty przyjmują barwę czerwoną/różową. Płytki krwi są ciemnofioletowe. Średnia wartość kanałów pozwala na skuteczną separację tych grup.

#### B. Analiza Kształtu (Shape Features)
Cechy geometryczne obliczono na podstawie masek binarnych. Kluczowe deskryptory to:
1.  **Współczynnik Okrągłości (Circularity):** Miarą podobieństwa obiektu do idealnego koła ($4\pi \cdot Area / Perimeter^2$). Pozwala na skuteczne rozróżnienie morfologiczne między gładkimi, regularnymi erytrocytami (RBC, okrągłość $\approx$ 1.0) a płytkami krwi (Platelets), które charakteryzują się nieregularnym, postrzępionym kształtem. Dodatkowo parametr ten wspomaga różnicowanie dużych leukocytów o nieregularnym obrysie cytoplazmy od idealnie owalnych erytrocytów.
2.  **Momenty Hu (Hu Moments):** Zbiór 7 niezmienników momentowych.
    * **Uzasadnienie:** Momenty Hu są niezmiennicze względem **translacji, skali i obrotu**. Jest to kluczowe w analizie mikroskopowej, gdzie orientacja komórki jest losowa. Wartości zostały zlogarytmowane w celu poprawy ich dynamiki i czytelności dla klasyfikatorów.
3.  **Obszar i Obwód:** Podstawowe cechy pozwalające na odróżnienie małych płytek krwi od dużych leukocytów.

> **Rozwiązanie problemu segmentacji RBC:** W funkcji analizy konturów zastosowano flagę `cv2.RETR_EXTERNAL`. Wymusza ona ignorowanie zagnieżdżonych konturów (dziur wewnątrz masek), co eliminuje błędy w obliczeniach obwodu dla krwinek czerwonych.

#### C. Analiza Tekstury (Texture Features)
Wykorzystano metodę **GLCM (Gray-Level Co-occurrence Matrix)** – Macierz Współwystępowania Poziomów Szarości.
* **Parametry:** Dystans = 1 piksel, Kąt = 0 stopni.
* **Wyznaczone cechy:**
    * *Contrast:* Miarą lokalnych zmian intensywności (ziarnistość cytoplazmy).
    * *Homogeneity:* Miarą jednolitości obszaru.
    * *Energy:* Miarą uporządkowania tekstury.
    * *Correlation:* Zależność liniowa między sąsiadującymi pikselami.

---

### 3. Implementacja Programowa

Poniższy kod realizuje iteracyjne przejście przez zbiór danych, ekstrakcję zdefiniowanych powyżej cech oraz zapis wyników do ustrukturyzowanego pliku CSV.

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops

# --- KONFIGURACJA ŚRODOWISKA ---
# Ścieżki do danych przetworzonych w Etapie 1
IMG_FOLDER = r'Output'
MASK_FOLDER = r'Output_masks'

# --- DEFINICJE FUNKCJI EKSTRAKCJI CECH ---

def get_color_features(img, mask):
    """
    Oblicza statystyki koloru wewnątrz obszaru zainteresowania (ROI).
    Input: Obraz BGR, Maska binarna.
    Output: Słownik ze średnimi wartościami kanałów B, G, R.
    """
    # Obliczenie średniej tylko dla pikseli należących do maski (mask > 0)
    # Pozwala to wyeliminować wpływ czarnego tła na statystyki koloru.
    mean_val = cv2.mean(img, mask=mask)
    return {
        'mean_b': mean_val[0],
        'mean_g': mean_val[1],
        'mean_r': mean_val[2]
    }

def get_shape_features(mask):
    """
    Ekstrakcja deskryptorów geometrycznych z maski binarnej.
    Implementuje zabezpieczenie przed artefaktami wewnątrz obiektów (np. dziury w RBC).
    """
    # cv2.RETR_EXTERNAL: Pobiera tylko zewnętrzny kontur obiektu.
    # Jest to kluczowe dla poprawności obliczeń obwodu w przypadku niedoskonałej segmentacji.
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    # Wybór największego konturu (eliminacja drobnego szumu)
    cnt = max(contours, key=cv2.contourArea)

    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    if perimeter == 0: return None

    # Obliczenie współczynnika okrągłości (Circularity)
    # Wartość bliska 1.0 oznacza idealne koło (np. limfocyty).
    circularity = (4 * np.pi * area) / (perimeter ** 2)

    # Obliczenie Momentów Hu (7 niezmienników)
    moments = cv2.moments(cnt)
    hu = cv2.HuMoments(moments).flatten()

    # Logarytmowanie momentów w celu normalizacji zakresu wartości
    # Zastosowano: -sgn(h) * log10(|h|)
    hu_log = []
    for h in hu:
        if h != 0:
            hu_log.append(-1 * np.sign(h) * np.log10(np.abs(h)))
        else:
            hu_log.append(0)

    return {
        'area': area,
        'perimeter': perimeter,
        'circularity': circularity,
        'hu1': hu_log[0], 'hu2': hu_log[1], 'hu3': hu_log[2],
        'hu4': hu_log[3], 'hu5': hu_log[4], 'hu6': hu_log[5], 'hu7': hu_log[6]
    }

def get_texture_features(img, mask):
    """
    Analiza tekstury przy użyciu macierzy GLCM (Gray-Level Co-occurrence Matrix).
    """
    # Konwersja do skali szarości jest wymagana dla GLCM
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Maskowanie tła (przypisanie wartości 0 pikselom spoza ROI)
    masked_gray = cv2.bitwise_and(gray, gray, mask=mask)

    # Obliczenie macierzy GLCM
    # distances=[1], angles=[0]: Analiza najbliższego sąsiada w poziomie
    glcm = graycomatrix(masked_gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

    # Ekstrakcja właściwości statystycznych z macierzy
    return {
        'texture_contrast': graycoprops(glcm, 'contrast')[0, 0],
        'texture_homogeneity': graycoprops(glcm, 'homogeneity')[0, 0],
        'texture_energy': graycoprops(glcm, 'energy')[0, 0],
        'texture_correlation': graycoprops(glcm, 'correlation')[0, 0]
    }

# --- PROCES GŁÓWNY (PIPELINE) ---

data_list = []

# Rekurencyjne przeszukiwanie struktury katalogów (obsługa podfolderów klas)
for root, dirs, files in os.walk(IMG_FOLDER):

    for filename in files:
        if not (filename.lower().endswith(('.jpg', '.png', '.jpeg'))):
            continue

        # Konstrukcja ścieżek
        img_path = os.path.join(root, filename)

        # Ustalenie ścieżki do odpowiadającej maski w równoległej strukturze katalogów
        relative_path = os.path.relpath(img_path, IMG_FOLDER)
        mask_path = os.path.join(MASK_FOLDER, relative_path)

        # Walidacja istnienia maski (z obsługą różnic w rozszerzeniach)
        if not os.path.exists(mask_path):
            mask_path_png = os.path.splitext(mask_path)[0] + ".png"
            if os.path.exists(mask_path_png):
                mask_path = mask_path_png
            else:
                continue # Pominięcie obrazu w przypadku braku maski

        # Wczytanie danych obrazowych
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if img is None or mask is None: continue

        # Ekstrakcja etykiety klasy z nazwy katalogu nadrzędnego
        label = os.path.basename(root)

        try:
            # 1. Ekstrakcja cech
            color_feats = get_color_features(img, mask)
            shape_feats = get_shape_features(mask)

            # Weryfikacja poprawności maski (czy obiekt został wykryty)
            if shape_feats is None: continue

            texture_feats = get_texture_features(img, mask)

            # 2. Agregacja danych
            features = {
                'filename': filename,
                'label': label
            }
            features.update(color_feats)
            features.update(shape_feats)
            features.update(texture_feats)

            data_list.append(features)

        except Exception as e:
            print(f"[ERROR] Błąd przetwarzania pliku {filename}: {e}")
            continue

# --- ZAPIS WYNIKÓW ---

df = pd.DataFrame(data_list)

if not df.empty:
    output_file = 'blood_cells_features.csv'
    df.to_csv(output_file, index=False)
    print(f"Proces zakończony pomyślnie.")
    print(f"Wygenerowano zbiór cech dla {len(df)} próbek.")
    print(f"Zidentyfikowane klasy: {df['label'].unique()}")
    print(f"Wyniki zapisano w: {output_file}")
else:
    print("Błąd: Nie wygenerowano żadnych danych. Sprawdź ścieżki wejściowe.")

Proces zakończony pomyślnie.
Wygenerowano zbiór cech dla 3448 próbek.
Zidentyfikowane klasy: ['Platelets' 'RBC' 'WBC']
Wyniki zapisano w: blood_cells_features.csv


### 4. Podsumowanie wyników

W wyniku działania skryptu wygenerowano plik `blood_cells_features.csv`.
Zbiór danych zawiera wygenerowane rekordy dla wszystkich poprawnie przetworzonych obrazów.

Każdy rekord opisany jest przez **17 cech numerycznych**:
* 3 cechy koloru (Mean B, G, R)
* 10 cech kształtu (Area, Perimeter, Circularity, 7 Momentów Hu)
* 4 cechy tekstury (Contrast, Homogeneity, Energy, Correlation)

Tak przygotowany zbiór danych jest gotowy do wykorzystania w Etapie 3 (Budowa i walidacja klasyfikatorów).